In [1]:
# Avoid errors when executing notebook.
Adam = Srini = Rami = Subby = Áron = None
chocolate = vanilla = pickles = None

# Backtracking Search

Many kinds of problems can only be solved using a trial-and-error approach. We usually have a set of constraints that help us decide if a proposed solution is valid. You saw some examples of this in lecture, for example the dinner invitation problem, or the n-queens problem. This is what we call backtracking search in computer science. We try a solution, and when we encounter an impossible situation in the process, we "bactrack" (go back to the last partial solution that worked) and try another solution.

## Boolean (Logical) Satisfiability

A large class of problems can be formulated as "logical equations," which can also be solved using backtracking search. There are many ways to state these problems, but we will adopt a common representation.

We have a set of variables that are either true of false, represented as keys in a dictionary. Let's introduce the operation NOT. This gives us literals, where a variable can either be in itself (_a_), or in a negated form (not *a*). We can combine these with the operation OR to get so-called "clauses." Finally, if we connect these clauses, we arrive at our final "formula."

In [2]:
clause1 = Adam or Srini
clause2 = not Rami or not Subby or not Áron
formula = (Adam or Srini) and (not Rami or not Subby or not Áron)

In [3]:
clause1 = [('Adam', True), ('Srini', True)]
clause2 = [('Rami', False), ('Subby', False), ('Áron', False)]
formula = [clause1, clause2]

### Gather all variables in the formula

In [4]:
def get_all_vars(formula):
    """Return a set of all variables in the provided logical formula."""
    return {var for clause in formula for var, boolean in clause}

In [5]:
get_all_vars(formula)

{'Adam', 'Rami', 'Srini', 'Subby', 'Áron'}

### Check if we have a solution

In [6]:
assignments1 = {
    'Adam': False,
    'Srini': True,
    'Rami': False,
}
assignments2 = {
    'Rami': True,
    'Subby': True,
    'Áron': True,
}

In [7]:
def is_full_solution(formula, assignments):
    """Return True if the assignments satisfy the formula, false otherwise."""
    for clause in formula:
        for var, boolean in clause:
            # if literal evaluates to True, clause evaluates to True
            if var in assignments and assignments[var] == boolean:
                break
            # otherwise, literal either isn't assigned or evaluates to False, so try next literal

        else:
            # no break: if we don't break out of the for loop,
            # none of the literals in the clause evaluated to True,
            # so the clause evaluates to False
            return False
    # if we get through the whole formula without returning False,
    # all the clauses must have evaluated to True, so the formula is True
    return True

    # Equivalent code
    #return all(any(var in assignments and assignments[var] == boolean for var, boolean in clause) for clause in formula)

In [8]:
is_full_solution(formula, assignments1)

True

In [9]:
is_full_solution(formula, assignments2)

False

### Find a solution

In [10]:
def satisfying_assignment(formula, assignments=None):
    """Return a variable assignment dictionary if the formula can be satisfied, None otherwise."""
    
    # Set up mutable default argument.
    if assignments is None:
        assignments = {}
    
    # If assignments satisfies the formula, return the assignments as they are.
    if is_full_solution(formula, assignments):
        return assignments
    
    all_variables = get_all_vars(formula)
    
    # If all variables have been assigned (but the assignment does not satisfy the formla), return None.
    if len(assignments) == len(all_variables):
        return None

    # Find the next unassigned variable, which we'll call test_var.
    unassigned_variables = all_variables - assignments.keys()
    test_var = unassigned_variables.pop()

    # High level idea: we can do a case analysis. there are only two cases:
    # test_var has to be either True or False (we can draw a tree of the cases to see this).

    # Approach: Try test_var True, then False for some unassigned variable.
    for boolean in (True, False):
        assignments_attempt = satisfying_assignment(formula, {**assignments, test_var: boolean})

        if assignments_attempt is not None and is_full_solution(formula, assignments_attempt):
            return assignments_attempt

    # Formula is unsatisfiable.
    return None


### Example problem: who ate the donuts?

The 6.009 staff were pleased to learn that grateful alumni had donated donuts for last week's staff meeting. Unfortunately, the donuts were gone when the staff showed up for the meeting! Who ate the donuts?
1. The suspects are Adam, Srini, Rami, Subby, and Áron.
1. Whichever suspect ate any of the donuts must have eaten all of them.
1. The donuts included exactly two of the flavors chocolate, vanilla, and pickles.
1. Adam only eats pickles-flavored donuts.
1. Years ago, Rami and Subby made a pact that, whenever either of them eats donuts, he must share with the other one.
1. Srini feels strongly about flavor fairness and will only eat donuts if he can include at least 3 different flavors.

In [11]:
rule1 = (Adam or Srini or Rami or Subby or Áron)
# At least one of them must have committed the crime!  Here, one of these
# variables being True represents that person having committed the crime.


rule2 = ((not Adam or not Srini)
     and (not Adam or not Rami)
     and (not Adam or not Subby)
     and (not Adam or not Áron)
     and (not Srini or not Rami)
     and (not Srini or not Subby)
     and (not Srini or not Áron)
     and (not Rami or not Subby)
     and (not Rami or not Áron)
     and (not Subby or not Áron))
# At most one of the suspects is guilty.  In other words, for any pair of
# suspects, at least one must be NOT guilty (so that we cannot possibly find
# two or more people guilty)
# Together, rule2 and rule1 guarantee that exactly one suspect is guilty.


rule3 = ((not chocolate or not vanilla or not pickles)
     and (chocolate or vanilla)
     and (chocolate or pickles)
     and (vanilla or pickles))
# Here is our rule that the cupcakes included exactly two of the flavors.  Put
# another way: it can't be that all flavors were present, and among any pair of
# flavors, at least one was present.


rule4 = ((not Adam or pickles)
     and (not Adam or not chocolate)
     and (not Adam or not vanilla))
# If Adam is guilty, this will evaluate to True only if only pickles-flavored
# cupcakes were present.  If Adam is not guilty, this will always evaluate to
# True.  This is our way of encoding the fact that, if Adam is guilty, only
# pickles-flavored cupcakes must have been present.


rule5 = (not Rami or Subby) and (not Subby or Rami)
# If Rami ate cupcakes without sharing with Subby, the first case will fail to
# hold.  Likewise for Subby eating without sharing.  Since Rami and Subby only
# eat cupcakes together, this rule excludes the possibility that only one of
# them ate cupcakes.


rule6 = ((not Srini or chocolate)
     and (not Srini or vanilla)
     and (not Srini or pickles))
# If Srini is the culprit and we left out a flavor, the corresponding case here
# will fail to hold.  So this rule encodes the restriction that Srini can only
# be guilty if all three types of cupcakes are present.


satisfied = rule1 and rule2 and rule3 and rule4 and rule5 and rule6

In [12]:
rule1 = [[('Adam', True), ('Srini', True), ('Rami', True),
          ('Subby', True), ('Áron', True)]]

rule2 = [[('Adam', False), ('Srini', False)],
         [('Adam', False), ('Rami', False)],
         [('Adam', False), ('Subby', False)],
         [('Adam', False), ('Áron', False)],
         [('Srini', False), ('Rami', False)],
         [('Srini', False), ('Subby', False)],
         [('Srini', False), ('Áron', False)],
         [('Rami', False), ('Subby', False)],
         [('Rami', False), ('Áron', False)],
         [('Subby', False), ('Áron', False)]]


rule3 = [[('chocolate', False), ('vanilla', False), ('pickles', False)],
         [('chocolate', True), ('vanilla', True)],
         [('chocolate', True), ('pickles', True)],
         [('vanilla', True), ('pickles', True)]]

rule4 = [[('Adam', False), ('pickles', True)],
         [('Adam', False), ('chocolate', False)],
         [('Adam', False), ('vanilla', False)]]

rule5 = [[('Rami', False), ('Subby', True)],
         [('Subby', False), ('Rami', True)]]

rule6 = [[('Srini', False), ('chocolate', True)],
         [('Srini', False), ('vanilla', True)],
         [('Srini', False), ('pickles', True)]]

rules = rule1 + rule2 + rule3 + rule4 + rule5 + rule6

In [13]:
satisfying_assignment(rules)

{'Subby': False,
 'Srini': False,
 'pickles': True,
 'Áron': True,
 'chocolate': True,
 'Rami': False,
 'Adam': False,
 'vanilla': False}